In [ ]:
# ===============================================
# ProgettoDL_FootballSegmentation – Sam su frame estratti
# Notebook: inferenza Sam su frame già presenti + ricomposizione video
# VERSIONE POTENZIATA E ADATTATA PER GOOGLE COLAB
# - Modello: ViT-Large (vit_l)
# - Modalità: Segment Everything (SamAutomaticMaskGenerator)
#
# Pipeline: (1) Setup ➔ (2) Percorsi ➔ (3) Caricamento modello ➔ (4) Selezione cartella frame
#           ➔ (5) Inferenza ➔ (6) Salvataggio frame + ricomposizione video ➔ (7) Report ➔ (8) Download
# ================================================


# ---------- (1) Setup e import librerie ----------
from pathlib import Path
import sys, re, subprocess, time, zipfile
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

# Installazione dipendenze se non presenti (utile per Colab)
try:
    import segment_anything
except ImportError:
    print("Installazione di Segment Anything...")
    subprocess.run([sys.executable, "-m", "pip", "install", "git+https://github.com/facebookresearch/segment-anything.git"], check=True)

try:
    from tqdm.notebook import tqdm
except ImportError:
    print("Installazione di tqdm...")
    subprocess.run([sys.executable, "-m", "pip", "install", "tqdm"], check=True)
    from tqdm.notebook import tqdm


# Controllo e setup repository Sam
sam_repo_path = Path("segment-anything")
!git --version

if not sam_repo_path.exists():
    print("🛠️ Clonazione del repository Segment Anything...")
    subprocess.run([
        "git", "clone",
        "https://github.com/facebookresearch/segment-anything",
        str(sam_repo_path)
    ], check=True)
else:
    print("✅ Repository Segment Anything già presente.")


# Aggiunge il percorso Sam per permettere l'import
sys.path.append(str(sam_repo_path.resolve()))

# Import Sam (Meta AI)
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

# Funzione ausiliaria per visualizzare le maschere (MODIFICATA per opacità regolabile)
def show_anns(anns, image, alpha=0.65):
    if len(anns) == 0:
        return image # Restituisce l'immagine originale se non ci sono maschere
    
    # Crea una copia dell'immagine su cui disegnare, convertita in float per il blending
    final_image = image.astype(np.float32) / 255.0
    
    # Ordina le maschere dalla più grande alla più piccola per disegnarle correttamente
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)

    for ann in sorted_anns:
        m = ann['segmentation']
        # Genera un colore casuale per la maschera
        color_mask = np.random.random(3)
        
        # Applica la maschera colorata all'immagine finale usando l'alpha specificato
        final_image[m] = final_image[m] * (1 - alpha) + color_mask * alpha

    # Riconverte l'immagine in formato 8-bit per il salvataggio
    return (final_image * 255).astype(np.uint8)


# ---------- (2) Definizione dei percorsi ----------
# In Colab, la directory di lavoro base è /content/
BASE_DIR = Path.cwd()
# Creiamo le cartelle necessarie dentro /content/
MODELS_DIR = BASE_DIR / "models"
DATA_OUTPUTS_DIR = BASE_DIR / "data_outputs"
MODELS_DIR.mkdir(parents=True, exist_ok=True)
DATA_OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

# Funzioni di utilità (incluse direttamente per semplicità)
def natural_sort(file_list):
    """Ordina una lista di file in modo 'naturale' (es. frame2.png prima di frame10.png)."""
    def convert(text):
        return int(text) if text.isdigit() else text.lower()
    def alphanum_key(key):
        return [convert(c) for c in re.split('([0-9]+)', str(key))]
    return sorted(file_list, key=alphanum_key)

def save_image(image_array, path):
    """Salva un'immagine da un array numpy."""
    # OpenCV si aspetta BGR, quindi se l'array è RGB, lo convertiamo
    if image_array.shape[2] == 3:
        image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
    cv2.imwrite(path, image_array)


# ---------- (3) Caricamento modello (Versione potenziata: ViT-Large) ----------

# Tipo di modello SAM (ViT-B → ViT-L)
sam_type = "vit_l"
model_filename = "sam_vit_l_0b3195.pth"
model_path = MODELS_DIR / model_filename

# URL ufficiale del checkpoint ViT-L
SAM_CHECKPOINT_URL = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth"

# Se il file .pth non esiste, scaricalo automaticamente
if model_path.exists():
    print(f"✅ Modello trovato: {model_path.name}")
else:
    print(f"⬇️ File dei pesi SAM non trovato. Download in corso (circa 1.2GB)...")
    try:
        import urllib.request
        # Mostra una barra di progresso per il download
        with tqdm(unit='B', unit_scale=True, miniters=1, desc=model_filename) as t:
            urllib.request.urlretrieve(SAM_CHECKPOINT_URL, model_path, reporthook=lambda b, bs, ts: t.update(b*bs - t.n))
        print("✅ Download completato.")
    except Exception as e:
        print(f"❌ Errore nel download dei pesi: {e}")
        # Rimuovi file parziale in caso di errore
        if model_path.exists():
            model_path.unlink()
        raise e

# Istanzia il modello SAM
sam = sam_model_registry[sam_type](checkpoint=str(model_path))

# Se disponibile CUDA, usa GPU; altrimenti CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Torch:", torch.__version__, "| CUDA:", torch.cuda.is_available(), "| Device:", device)
if torch.cuda.is_available():
    print("GPU usata:", torch.cuda.get_device_name(0))

# Sposta il modello sul device
sam.to(device)

# Inizializza il generatore automatico di maschere
# NOTA: Puoi personalizzare i parametri qui per migliorare la qualità vs velocità
mask_generator = SamAutomaticMaskGenerator(sam)


# ---------- (4) Selezione cartella frame da analizzare (Upload via Colab) ----------
try:
    from google.colab import files
    # Chiede all'utente di caricare un file ZIP contenente i frame
    print("Per favore, crea un file ZIP con la cartella dei frame e selezionalo tramite il pulsante qui sotto.")
    uploaded = files.upload()

    if not uploaded:
        raise ValueError("Nessun file caricato. Esecuzione interrotta.")

    # Prende il nome del primo file caricato
    zip_name = list(uploaded.keys())[0]
    zip_path = Path(zip_name)

    # Definisce la cartella di destinazione per l'estrazione
    extract_dir = BASE_DIR / "extracted_frames_from_zip"
    if extract_dir.exists():
        import shutil
        shutil.rmtree(extract_dir) # Pulisce la cartella se esiste già
    extract_dir.mkdir(parents=True, exist_ok=True)


    print(f"Estrazione di '{zip_name}' in corso...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print("Estrazione completata.")

    # Cerca la cartella contenente i frame all'interno della directory estratta.
    # Spesso, lo zip contiene una cartella radice.
    # Troviamo la prima sottocartella che contiene file .png
    found_frames_dir = None
    for path_object in extract_dir.rglob('*.png'):
        found_frames_dir = path_object.parent
        break # Trovata la prima cartella con PNG, usiamo quella

    if not found_frames_dir:
         # Se non ci sono sottocartelle, i frame potrebbero essere nella root
        if list(extract_dir.glob('*.png')):
            found_frames_dir = extract_dir
        else:
            raise FileNotFoundError(f"Nessun file PNG trovato nel file ZIP estratto in: {extract_dir}")

    frames_dir = found_frames_dir
    print(f"Cartella dei frame impostata su: {frames_dir}")

    # Rimuove il file zip per pulire lo spazio
    zip_path.unlink()

except ImportError:
    # Fallback per ambienti non-Colab
    print("Ambiente non Colab rilevato. Imposta manualmente 'frames_dir'.")
    # Inserisci qui il percorso locale se non usi Colab
    frames_dir = Path("percorso/locale/alla/tua/cartella_frames")


if not frames_dir.exists() or not frames_dir.is_dir():
    raise FileNotFoundError(f"Cartella frame non trovata o non valida: '{frames_dir}'.")

# Prende tutti i frame PNG con prefisso 'frame' e li ordina in modo naturale
frame_paths = natural_sort(list(frames_dir.glob("frame*.png")))

if not frame_paths:
    raise RuntimeError(f"Nessun frame PNG trovato in {frames_dir}")

# Estrai automaticamente l'ID clip (xx) dal nome cartella (es. .../extracted_frames_clip01). Default '00'
m = re.search(r"clip(\d+)", frames_dir.name)
clip_id = m.group(1) if m else "00"
print(f"Trovati {len(frame_paths)} frame per la clip ID: {clip_id}")


# ---------- (5) Inferenza SAM (Segment Everything) sui frame ----------
# FPS fisso per la ricostruzione del video
fps = 25.0

# !!! NUOVA IMPOSTAZIONE: REGOLA L'OPACITÀ DELLA MASCHERA QUI !!!
# 1.0 = completamente opaca (colori pieni), 0.3 = molto trasparente
MASK_OPACITY = 0.5

# Cartella per i frame segmentati in /content/data_outputs:
# "segmented_frames_clipxx_sam"
seg_frames_dir = DATA_OUTPUTS_DIR / f"segmented_frames_clip{clip_id}_sam"
seg_frames_dir.mkdir(parents=True, exist_ok=True)

segmented_frames = []  # terrà gli array RGB annotati, nell'ordine corretto
processed_names = []   # lista dinamica mostrata sotto la barra

# Display dinamico della lista sotto la barra (mostriamo gli ultimi 50 file)
list_handle = display(Markdown("_Inizio inferenza..._"), display_id=True)

# (Aggiunta timing totale inferenza)
start_time = time.time()  # Per misurare la durata totale dell'inferenza

for i, frame_path in tqdm(list(enumerate(frame_paths, start=1)), desc=f"Inferenza SAM 'Everything' (clip {clip_id})", unit="frame"):
    # Carica frame e converte in RGB
    frame_bgr = cv2.imread(str(frame_path))
    if frame_bgr is None:
        print(f"Attenzione: impossibile leggere il frame {frame_path}")
        continue
    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)

    # Genera tutte le maschere per l'immagine corrente
    masks = mask_generator.generate(frame_rgb)

    # Applica overlay delle maschere sull’immagine usando l'opacità definita
    annotated = show_anns(masks, frame_rgb, alpha=MASK_OPACITY)

    # Salvataggio frame segmentato
    seg_name = f"frame{i:04d}_clip{clip_id}_segmented_sam.png"
    seg_path = seg_frames_dir / seg_name
    save_image(annotated, str(seg_path))

    # Accumula in memoria per ricostruzione video
    segmented_frames.append(annotated)

    # Aggiorna lista dinamica (ultimi 10)
    processed_names.append(seg_name)
    last = processed_names[-10:]
    list_handle.update(Markdown(
        "**Frame analizzati (ultimi 10):** \n" + "<br>".join(last) + f"<br><br>Totale: **{len(processed_names)}**"
    ))

if not segmented_frames:
    raise RuntimeError("Nessun frame segmentato generato.")

elapsed_time = time.time() - start_time


# ---------- (6) Ricomposizione video (barra + lista dinamica coerenti) ----------
# Salva il video in /content/data_outputs come "segmented_clipxx_sam.mp4"
out_video_path = DATA_OUTPUTS_DIR / f"segmented_clip{clip_id}_sam.mp4"

# Inizializza VideoWriter con risoluzione del primo frame
h, w = segmented_frames[0].shape[:2]
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
writer = cv2.VideoWriter(str(out_video_path), fourcc, fps, (w, h))

written_names = []  # per la lista dinamica dei frame inseriti
recon_handle = display(Markdown("_Inizio ricostruzione video…_"), display_id=True)

for i, frame in tqdm(list(enumerate(segmented_frames, start=1)), desc="Ricostruzione video", unit="frame"):
    # Scrive il frame nel video (convertendo da RGB a BGR per OpenCV)
    writer.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

    # Aggiorna lista dinamica (ultimi 10)
    name = f"frame{i:04d}_clip{clip_id}_segmented_sam.png"
    written_names.append(name)
    last_w = written_names[-10:]
    recon_handle.update(Markdown(
        "**Frame inseriti nel video (ultimi 10):** \n" + "<br>".join(last_w) + f"<br><br>Totale: **{len(written_names)}**"
    ))

writer.release()


# ---------- (7) Report finale ----------

fps_effettivo = len(segmented_frames) / elapsed_time if elapsed_time > 0 else 0

print(
    "\n✅ Segmentazione SAM (Segment Everything) avvenuta con successo.\n"
    
    "\n🔹 ----------------- Caratteristiche modello / architettura utilizzata -----------------\n"
    f"- Cartella notebook eseguito: {BASE_DIR}\n"
    f"- Architettura SAM: Segment Anything\n"
    f"- Tipo architettura ViT: {sam_type.upper()}\n"
    f"- Modello SAM utilizzato: {model_path.name} (cartella: {MODELS_DIR})\n"
    f"- Tipo di prompt utilizzato: Automatic Mask Generation ('Segment Everything')\n"

    "\n🔹 ----------------- Caratteristiche dell’input dell’inferenza -----------------\n"
    f"- Segmentazione relativa a: clip{clip_id}.mp4\n"
    f"- FPS della clip{clip_id}: {fps} fps\n"
    f"- Durata della clip {clip_id}: {len(segmented_frames)/fps:.3f} secondi\n"
    f"- Cartella frames estratti da clip{clip_id}: {frames_dir}\n"
    f"- Risoluzione singoli frames: {w}x{h} pixel\n"

    "\n🔹 ----------------- Caratteristiche dell’output dell’inferenza -----------------\n"
    f"- Cartella frames segmentati (PNG): {seg_frames_dir}\n"
    f"- Video segmentato ricostruito (MP4): {out_video_path}\n"

    "\n🔹 ----------------- Analisi inferenza -----------------\n"
    f"- Numero totale di frame segmentati: {len(segmented_frames)} frames\n"
    f"- Tempo totale di inferenza: {elapsed_time:.2f} secondi\n"
    f"- FPS effettivi di inferenza: {fps_effettivo:.2f} frame/secondo\n"
    f"- Tempo di inferenza medio per ciascun frame: {1/fps_effettivo:.3f} secondi/frame\n"
)

# =======================================
# Fine notebook SAM (Segment Everything)
# =======================================


# ---------- (8) Download dei risultati in un file ZIP ----------
try:
    import shutil
    from google.colab import files

    # Nome del file ZIP che conterrà tutti gli output, includendo modello e clip ID
    output_zip_name = f"output_SAM_{sam_type}_clip{clip_id}.zip"

    # Cartella da comprimere (la cartella principale che contiene sia i frame sia il video)
    folder_to_zip = DATA_OUTPUTS_DIR

    print(f"\nCompressing '{folder_to_zip}' into '{output_zip_name}'...")

    # Crea l'archivio ZIP
    # shutil.make_archive('nome_file_senza_estensione', 'formato', 'cartella_da_zippare')
    shutil.make_archive(output_zip_name.replace('.zip', ''), 'zip', folder_to_zip)

    print(f"Archive created! Starting download of '{output_zip_name}'...")

    # Avvia il download del file ZIP nel tuo browser
    files.download(output_zip_name)

except ImportError:
    print("\nDownload non disponibile: eseguire in un ambiente Google Colab per scaricare i risultati.")
except NameError:
    print("\nErrore: impossibile generare il file ZIP. Assicurarsi che le celle precedenti siano state eseguite correttamente.")